#Prepared by Tamal Acharya

In [ ]:
#Tutorial on model context protocol

# Define the structure for the context
# For a running sum, the context is just the current sum
class RunningSumContext:
  def __init__(self, current_sum=0):
    self.current_sum = current_sum

# Implement the processor adhering to our conceptual protocol
class RunningSumProcessor:
  def initialize_context(self):
    """Initializes the context for a new sum."""
    return RunningSumContext(current_sum=0)

  def process(self, input_value, context: RunningSumContext):
    """
    Adds the input_value to the context's sum.

    Returns:
        tuple: (result, updated_context)
    """
    # The "result" here is the new sum
    new_sum = context.current_sum + input_value
    # The updated context contains the new sum
    updated_context = RunningSumContext(current_sum=new_sum)

    return new_sum, updated_context

  def get_result(self, process_output, context):
    """In this case, the process output is the result."""
    return process_output

# --- Demonstration ---

# Instantiate the processor
processor = RunningSumProcessor()

# Initialize the context
current_context = processor.initialize_context()
print(f"Initial context sum: {current_context.current_sum}")

# Process inputs step by step, updating the context each time
inputs = [10, 5, -2, 8]
results = []

print("\nProcessing steps:")
for i, value in enumerate(inputs):
  print(f"  Processing input: {value}")
  result, current_context = processor.process(value, current_context)
  results.append(processor.get_result(result, current_context)) # Use get_result to get the final step output
  print(f"    Result (current sum): {result}")
  print(f"    Updated context sum: {current_context.current_sum}")


print(f"\nFinal results at each step: {results}")
print(f"Final total sum: {current_context.current_sum}") # Access sum from final context


In [ ]:
#
# Install necessary libraries
# !pip install transformers datasets
# !pip install faiss-cpu # For efficient similarity search
# !pip install openai # If you're using OpenAI models


In [ ]:
#
# Tutorial on Model Context Protocol (MCP) with RAG and Agentic AI

# **1. Introduction to Model Context Protocol (MCP)**

# The Model Context Protocol (MCP) is not a formally defined standard but rather a conceptual pattern
# for managing the state or "context" of a sophisticated AI process, especially in scenarios
# involving multiple steps, interactions, or the integration of external information.

# Key ideas behind MCP:
# - **State Management:** How does the AI process remember relevant information from previous steps?
# - **Context Object:** A dedicated structure (often a class or dictionary) to hold the current state.
# - **Processors:** Modules or functions that take the current context and new input, perform an operation,
#   and return a result along with an *updated* context.
# - **Separation of Concerns:** The processing logic is separate from the context state.
# - **Extensibility:** Allows for complex workflows where different processors update the same context.

# **Why is this important for RAG and Agentic AI?**

# - **RAG (Retrieval Augmented Generation):** Needs to remember the retrieved documents and the user's query
#   to generate a coherent response. The context can hold the query, retrieved documents, and intermediate generated text.
# - **Agentic AI:** Agents perform sequences of actions (e.g., searching, analyzing, planning). They need to
#   maintain a "working memory" or context that tracks the goal, performed actions, observations, and intermediate conclusions.
#   MCP provides a structure for this working memory.

# **2. Implementing MCP for a RAG Agent**

# Let's build a simple RAG agent that can answer questions based on a small corpus.
# The agent will:
# 1. Take a user query.
# 2. Retrieve relevant documents from a corpus.
# 3. Use a language model to generate an answer based on the query and retrieved documents.

# The context will hold:
# - The original query
# - The retrieved documents
# - The generated answer

# **Define the Context Structure**
# The context needs to hold the state of our RAG process.
class RagContext:
  def __init__(self, query=None, retrieved_docs=None, generated_answer=None):
    self.query = query # The initial user query
    self.retrieved_docs = retrieved_docs if retrieved_docs is not None else [] # List of retrieved documents
    self.generated_answer = generated_answer # The final answer

  def __repr__(self):
    return f"RagContext(query='{self.query}', num_docs={len(self.retrieved_docs)}, answer_set={self.generated_answer is not None})"


# **Define the Processors**
# We'll have two main processors: one for retrieval and one for generation.

# **Processor 1: Retrieval**
# This processor takes the query from the context and retrieves relevant documents.
# For simplicity, we'll use a basic keyword search. In a real RAG system, you'd use vector embeddings and similarity search (like FAISS).

# Dummy corpus
corpus = [
    {"id": 1, "text": "The capital of France is Paris."},
    {"id": 2, "text": "Paris is famous for the Eiffel Tower."},
    {"id": 3, "text": "The Mona Lisa is in the Louvre Museum in Paris."},
    {"id": 4, "text": "Rome is the capital of Italy."},
    {"id": 5, "text": "The Colosseum is in Rome."},
]

class RetrievalProcessor:
  def __init__(self, corpus):
    self.corpus = corpus

  def process(self, input_value, context: RagContext):
    """
    Retrieves documents based on the query in the context.
    input_value is not used here, the query comes from the context.
    """
    if context.query is None:
      raise ValueError("Context must contain a query for retrieval.")

    # Simple keyword search (replace with vector search in production)
    query_keywords = context.query.lower().split()
    retrieved_docs = []
    for doc in self.corpus:
      if any(keyword in doc['text'].lower() for keyword in query_keywords):
        retrieved_docs.append(doc)

    # The "result" of this step could be the retrieved docs, but we primarily update the context
    result = retrieved_docs

    # Update the context with retrieved documents
    updated_context = RagContext(
        query=context.query,
        retrieved_docs=retrieved_docs,
        generated_answer=context.generated_answer # Keep existing answer if any
    )

    return result, updated_context

  def get_result(self, process_output, context):
      """The result of this step is the list of retrieved documents."""
      return process_output


# **Processor 2: Generation**
# This processor takes the query and retrieved documents from the context and generates an answer.
# We'll use a placeholder function for the language model.

class GenerationProcessor:
  def __init__(self):
    # In a real scenario, you would load a language model here (e.g., from transformers library or OpenAI API)
    pass

  def generate_answer_with_llm(self, query, documents):
    """
    Placeholder for LLM call.
    In reality, you'd format the prompt with query and documents and call an API/model.
    """
    doc_texts = "\n".join([doc['text'] for doc in documents])
    if not doc_texts:
        return "Could not find relevant information."

    # Simple placeholder logic
    if "capital of France" in query.lower() and "Paris" in doc_texts:
        return "Based on the information, the capital of France is Paris."
    elif "Eiffel Tower" in query.lower() and "Paris" in doc_texts:
        return "The Eiffel Tower is in Paris."
    elif "Mona Lisa" in query.lower() and "Louvre" in doc_texts and "Paris" in doc_texts:
        return "The Mona Lisa is in the Louvre Museum in Paris."
    elif "capital of Italy" in query.lower() and "Rome" in doc_texts:
         return "Based on the information, the capital of Italy is Rome."
    else:
        return f"Based on the provided information:\n{doc_texts}\n\nAnswer to '{query}': This is a placeholder answer based on the docs."


  def process(self, input_value, context: RagContext):
    """
    Generates an answer based on the query and retrieved documents in the context.
    input_value is not used here.
    """
    if context.query is None or not context.retrieved_docs:
      # We can still attempt generation, but it might be less effective
      print("Warning: Generating answer without query or retrieved docs in context.")


    # Generate the answer using the LLM placeholder
    generated_answer = self.generate_answer_with_llm(context.query, context.retrieved_docs)

    # The "result" of this step is the generated answer
    result = generated_answer

    # Update the context with the generated answer
    updated_context = RagContext(
        query=context.query,
        retrieved_docs=context.retrieved_docs, # Keep retrieved docs
        generated_answer=generated_answer
    )

    return result, updated_context

  def get_result(self, process_output, context):
      """The result of this step is the generated answer."""
      return process_output


# **3. Building the Agent Workflow using MCP**

# An agent orchestrates the execution of different processors, managing the context flow between them.

class RagAgent:
  def __init__(self, retrieval_processor, generation_processor):
    self.retrieval_processor = retrieval_processor
    self.generation_processor = generation_processor
    self.current_context = None # The agent holds the persistent context

  def initialize_session(self):
    """Starts a new interaction session by resetting the context."""
    self.current_context = RagContext()
    print("Agent session initialized.")

  def ask_question(self, query):
    """Processes a user question using the RAG workflow."""
    if self.current_context is None:
        print("No active session. Initializing a new one.")
        self.initialize_session()

    # --- Step 1: Set the query in the context ---
    # This is a bit different - the *agent* updates the context directly to start.
    # Or, you could have a separate "Query Setter" processor. Let's do direct for simplicity.
    self.current_context.query = query
    print(f"\nAgent received query: '{query}'")
    print(f"Current context after setting query: {self.current_context}")

    # --- Step 2: Perform Retrieval ---
    print("\nAgent performing retrieval...")
    retrieval_result, updated_context_after_retrieval = self.retrieval_processor.process(None, self.current_context)
    self.current_context = updated_context_after_retrieval # Update agent's context
    retrieved_docs = self.retrieval_processor.get_result(retrieval_result, self.current_context)
    print(f"Retrieval complete. Found {len(retrieved_docs)} documents.")
    # print(f"Retrieved Docs: {retrieved_docs}") # Uncomment to see docs
    print(f"Current context after retrieval: {self.current_context}")


    # --- Step 3: Perform Generation ---
    print("\nAgent performing generation...")
    generation_result, updated_context_after_generation = self.generation_processor.process(None, self.current_context)
    self.current_context = updated_context_after_generation # Update agent's context
    final_answer = self.generation_processor.get_result(generation_result, self.current_context)
    print("Generation complete.")
    print(f"Current context after generation: {self.current_context}")


    # --- Step 4: Provide the Final Result ---
    print("\n--- Final Answer ---")
    print(final_answer)
    print("--------------------\n")

    return final_answer

  def get_current_context(self):
      return self.current_context


# **4. Running the RAG Agent Example**

# Instantiate the processors
retrieval_proc = RetrievalProcessor(corpus=corpus)
generation_proc = GenerationProcessor() # Placeholder LLM

# Instantiate the agent
rag_agent = RagAgent(retrieval_processor=retrieval_proc, generation_processor=generation_proc)

# Initialize a session
rag_agent.initialize_session()

# Ask a question
rag_agent.ask_question("What is the capital of France?")

# Ask another question - the context was reset in initialize_session
rag_agent.ask_question("Where is the Eiffel Tower?")

# Let's try asking a question that uses information from multiple documents or is slightly outside the simple rules
# rag_agent.ask_question("Tell me about the Mona Lisa location.")

# Examine the context after the last question
print("\nAgent's final context after last question:")
print(rag_agent.get_current_context())


# **5. Extending to More Complex Agentic Workflows**

# The MCP pattern is even more powerful for complex agents that might:
# - Use multiple tools (web search, calculator, API calls)
# - Engage in multi-turn conversations
# - Plan sequences of actions
# - Reflect on previous steps

# **Example: A Simple Multi-step Agent**

# Let's imagine an agent that needs to find information and then potentially perform a calculation.

# Define a new context that can hold more information
class MultiStepContext:
    def __init__(self, query=None, info_found=None, calculated_value=None, current_task="find_info"):
        self.query = query
        self.info_found = info_found # e.g., numbers extracted from text
        self.calculated_value = calculated_value
        self.current_task = current_task # State variable to guide the agent's next step

    def __repr__(self):
        return f"MultiStepContext(query='{self.query}', task='{self.current_task}', info_found={self.info_found}, calculated={self.calculated_value is not None})"

# Define a processor to find information (similar to Retrieval, but could extract specific data)
class InfoFinderProcessor:
    def process(self, input_value, context: MultiStepContext):
        """Simulates finding numerical information related to the query."""
        if context.query is None:
            return None, context # Cannot proceed without a query

        found_info = None
        # Dummy logic: if query asks about "items" and mentions a number
        import re
        match = re.search(r'how many (\w+) are there with (\d+)', context.query)
        if match:
            item_type = match.group(1)
            number = int(match.group(2))
            print(f"  InfoFinder: Found query about '{item_type}' with number '{number}'.")
            # Simulate finding related info - let's say we find another number
            found_info = [number, number * 2] # Example: found the input number and double it

        # Update context
        updated_context = MultiStepContext(
            query=context.query,
            info_found=found_info,
            calculated_value=context.calculated_value,
            current_task="calculate" if found_info is not None else "failed_info_find" # Update task based on finding info
        )
        return found_info, updated_context # Result of this step is the found info

    def get_result(self, process_output, context):
        return process_output

# Define a processor to perform a calculation
class CalculatorProcessor:
    def process(self, input_value, context: MultiStepContext):
        """Simulates a calculation based on info_found in the context."""
        if context.info_found is None or not isinstance(context.info_found, list) or len(context.info_found) < 2:
            print("  Calculator: Not enough information to calculate.")
            return None, context # Cannot calculate

        try:
            num1 = context.info_found[0]
            num2 = context.info_found[1]
            calculated_value = num1 + num2 # Simple calculation
            print(f"  Calculator: Calculating {num1} + {num2} = {calculated_value}")

            # Update context
            updated_context = MultiStepContext(
                query=context.query,
                info_found=context.info_found,
                calculated_value=calculated_value,
                current_task="report_result" # Move to reporting stage
            )
            return calculated_value, updated_context # Result is the calculated value

        except Exception as e:
            print(f"  Calculator Error: {e}")
            # Update context to reflect failure
            updated_context = MultiStepContext(
                query=context.query,
                info_found=context.info_found,
                calculated_value=None,
                current_task="calculation_failed"
            )
            return None, updated_context

    def get_result(self, process_output, context):
        return process_output

# Define a processor to report the final result
class ReporterProcessor:
    def process(self, input_value, context: MultiStepContext):
        """Generates a final report based on the context state."""
        final_report = "Processing complete."
        if context.calculated_value is not None:
            final_report = f"Based on your query, the calculated value is: {context.calculated_value}"
        elif context.info_found is not None:
             final_report = f"Found some information but couldn't complete the calculation. Information found: {context.info_found}"
        else:
             final_report = "Could not find relevant information or perform calculation."

        # Context remains the same for the final reporting step, or you could add a 'finished' flag
        updated_context = MultiStepContext(
             query=context.query,
             info_found=context.info_found,
             calculated_value=context.calculated_value,
             current_task="finished" # Mark as finished
        )

        return final_report, updated_context # Result is the final report

    def get_result(self, process_output, context):
        return process_output


# Build a simple Multi-step Agent orchestrator
class MultiStepAgent:
    def __init__(self, info_finder, calculator, reporter):
        self.info_finder = info_finder
        self.calculator = calculator
        self.reporter = reporter
        self.current_context = None

    def initialize_session(self, query):
        """Starts a new multi-step session."""
        self.current_context = MultiStepContext(query=query, current_task="find_info")
        print(f"Agent session initialized for query: '{query}'")
        print(f"Initial context: {self.current_context}")

    def run_next_step(self):
        """Runs the next appropriate processor based on the current context's task."""
        if self.current_context is None:
            print("Agent not initialized. Call initialize_session first.")
            return None

        current_task = self.current_context.current_task
        print(f"\nAgent executing step: {current_task}")

        result = None
        updated_context = self.current_context # Default to no change if task not matched

        if current_task == "find_info":
            result, updated_context = self.info_finder.process(None, self.current_context)
            step_result = self.info_finder.get_result(result, updated_context)
            print(f"  Step result (Info Finder): {step_result}")

        elif current_task == "calculate":
             result, updated_context = self.calculator.process(None, self.current_context)
             step_result = self.calculator.get_result(result, updated_context)
             print(f"  Step result (Calculator): {step_result}")

        elif current_task == "report_result" or current_task == "failed_info_find" or current_task == "calculation_failed":
             result, updated_context = self.reporter.process(None, self.current_context)
             step_result = self.reporter.get_result(result, updated_context)
             print(f"  Step result (Reporter): {step_result}")
             print("\n--- Multi-step Process Finished ---")
             print(f"Final Outcome: {step_result}")
             # We might not update the context if it's the final step, or mark it finished
             self.current_context = updated_context # Update context to mark 'finished'
             return step_result # Return the final result

        else:
            print(f"Agent reached an unknown task state: {current_task}")
            # You might want to handle errors or loop detection here

        self.current_context = updated_context # Update the agent's context for the next step
        print(f"Context after step: {self.current_context}")

        # Return the step result and indicate if finished
        return step_result # Return result of the just-completed step


    def run_until_finished(self):
        """Runs the agent step-by-step until it reaches a 'finished' state."""
        if self.current_context is None:
            print("Agent not initialized. Call initialize_session first.")
            return None

        last_result = None
        while self.current_context.current_task != "finished" and \
              "failed" not in self.current_context.current_task: # Also stop on failure states
            step_output = self.run_next_step()
            if self.current_context.current_task == "finished" or "failed" in self.current_context.current_task:
                 last_result = step_output # Capture the final output
                 break # Exit loop if finished or failed
            # Optional: Add a safety break for infinite loops
            # step_counter += 1
            # if step_counter > MAX_STEPS: break


        if self.current_context.current_task == "finished":
             print("\nAgent successfully finished.")
             return last_result # Return the final reported result
        else:
             print(f"\nAgent finished in state: {self.current_context.current_task}")
             if last_result:
                 print(f"Last step output: {last_result}")
             return None # Indicate failure or incomplete state


# **6. Running the Multi-step Agent Example**

# Instantiate processors
info_finder_proc = InfoFinderProcessor()
calculator_proc = CalculatorProcessor()
reporter_proc = ReporterProcessor()

# Instantiate the agent
multi_step_agent = MultiStepAgent(info_finder=info_finder_proc, calculator=calculator_proc, reporter=reporter_proc)

# Run a query through the agent workflow
query1 = "Can you calculate the sum of two numbers if there are 5 items and 10 items?"
multi_step_agent.initialize_session(query1)
final_output1 = multi_step_agent.run_until_finished()

print("\n--- End of Agent Run 1 ---\n")
print(f"Final context: {multi_step_agent.get_current_context()}")


# Run another query that might not trigger the calculation step
query2 = "Tell me about cats."
multi_step_agent.initialize_session(query2)
final_output2 = multi_step_agent.run_until_finished()

print("\n--- End of Agent Run 2 ---\n")
print(f"Final context: {multi_step_agent.get_current_context()}")


# **7. Conclusion**

# The Model Context Protocol (MCP), used here as a conceptual pattern, provides a robust way to structure
# complex AI workflows, particularly for RAG and agentic systems.

# By defining a clear context object and modular processors that update this context, we gain:
# - **Modularity:** Processors are self-contained and reusable.
# - **Statefulness:** The context preserves information across steps.
# - **Observability:** We can inspect the context at any point to understand the agent's state.
# - **Flexibility:** Easily add or change processors and orchestrate complex sequences of operations.

# This pattern is foundational for building sophisticated AI agents that can interact,
# use tools, and maintain coherence over multiple steps.


In [ ]:
#
import re

# Define the structure for the context
# For a running sum, the context is just the current sum
class RunningSumContext:
  def __init__(self, current_sum=0):
    self.current_sum = current_sum

# Implement the processor adhering to our conceptual protocol
class RunningSumProcessor:
  def initialize_context(self):
    """Initializes the context for a new sum."""
    return RunningSumContext(current_sum=0)

  def process(self, input_value, context: RunningSumContext):
    """
    Adds the input_value to the context's sum.

    Returns:
        tuple: (result, updated_context)
    """
    # The "result" here is the new sum
    new_sum = context.current_sum + input_value
    # The updated context contains the new sum
    updated_context = RunningSumContext(current_sum=new_sum)

    return new_sum, updated_context

  def get_result(self, process_output, context):
    """In this case, the process output is the result."""
    return process_output

# --- Demonstration ---

# Instantiate the processor
processor = RunningSumProcessor()

# Initialize the context
current_context = processor.initialize_context()
print(f"Initial context sum: {current_context.current_sum}")

# Process inputs step by step, updating the context each time
inputs = [10, 5, -2, 8]
results = []

print("\nProcessing steps:")
for i, value in enumerate(inputs):
  print(f"  Processing input: {value}")
  result, current_context = processor.process(value, current_context)
  results.append(processor.get_result(result, current_context)) # Use get_result to get the final step output
  print(f"    Result (current sum): {result}")
  print(f"    Updated context sum: {current_context.current_sum}")


print(f"\nFinal results at each step: {results}")
print(f"Final total sum: {current_context.current_sum}") # Access sum from final context


# Install necessary libraries
!pip install transformers datasets
!pip install faiss-cpu # For efficient similarity search
!pip install openai # If you're using OpenAI models


# Tutorial on Model Context Protocol (MCP) with RAG and Agentic AI

# **1. Introduction to Model Context Protocol (MCP)**

# The Model Context Protocol (MCP) is not a formally defined standard but rather a conceptual pattern
# for managing the state or "context" of a sophisticated AI process, especially in scenarios
# involving multiple steps, interactions, or the integration of external information.

# Key ideas behind MCP:
# - **State Management:** How does the AI process remember relevant information from previous steps?
# - **Context Object:** A dedicated structure (often a class or dictionary) to hold the current state.
# - **Processors:** Modules or functions that take the current context and new input, perform an operation,
#   and return a result along with an *updated* context.
# - **Separation of Concerns:** The processing logic is separate from the context state.
# - **Extensibility:** Allows for complex workflows where different processors update the same context.

# **Why is this important for RAG and Agentic AI?**

# - **RAG (Retrieval Augmented Generation):** Needs to remember the retrieved documents and the user's query
#   to generate a coherent response. The context can hold the query, retrieved documents, and intermediate generated text.
# - **Agentic AI:** Agents perform sequences of actions (e.g., searching, analyzing, planning). They need to
#   maintain a "working memory" or context that tracks the goal, performed actions, observations, and intermediate conclusions.
#   MCP provides a structure for this working memory.

# **2. Implementing MCP for a RAG Agent**

# Let's build a simple RAG agent that can answer questions based on a small corpus.
# The agent will:
# 1. Take a user query.
# 2. Retrieve relevant documents from a corpus.
# 3. Use a language model to generate an answer based on the query and retrieved documents.

# The context will hold:
# - The original query
# - The retrieved documents
# - The generated answer

# **Define the Context Structure**
# The context needs to hold the state of our RAG process.
class RagContext:
  def __init__(self, query=None, retrieved_docs=None, generated_answer=None):
    self.query = query # The initial user query
    self.retrieved_docs = retrieved_docs if retrieved_docs is not None else [] # List of retrieved documents
    self.generated_answer = generated_answer # The final answer

  def __repr__(self):
    return f"RagContext(query='{self.query}', num_docs={len(self.retrieved_docs)}, answer_set={self.generated_answer is not None})"


# **Define the Processors**
# We'll have two main processors: one for retrieval and one for generation.

# **Processor 1: Retrieval**
# This processor takes the query from the context and retrieves relevant documents.
# For simplicity, we'll use a basic keyword search. In a real RAG system, you'd use vector embeddings and similarity search (like FAISS).

# Dummy corpus
corpus = [
    {"id": 1, "text": "The capital of France is Paris."},
    {"id": 2, "text": "Paris is famous for the Eiffel Tower."},
    {"id": 3, "text": "The Mona Lisa is in the Louvre Museum in Paris."},
    {"id": 4, "text": "Rome is the capital of Italy."},
    {"id": 5, "text": "The Colosseum is in Rome."},
]

class RetrievalProcessor:
  def __init__(self, corpus):
    self.corpus = corpus

  def process(self, input_value, context: RagContext):
    """
    Retrieves documents based on the query in the context.
    input_value is not used here, the query comes from the context.
    """
    if context.query is None:
      raise ValueError("Context must contain a query for retrieval.")

    # Simple keyword search (replace with vector search in production)
    query_keywords = context.query.lower().split()
    retrieved_docs = []
    for doc in self.corpus:
      if any(keyword in doc['text'].lower() for keyword in query_keywords):
        retrieved_docs.append(doc)

    # The "result" of this step could be the retrieved docs, but we primarily update the context
    result = retrieved_docs

    # Update the context with retrieved documents
    updated_context = RagContext(
        query=context.query,
        retrieved_docs=retrieved_docs,
        generated_answer=context.generated_answer # Keep existing answer if any
    )

    return result, updated_context

  def get_result(self, process_output, context):
      """The result of this step is the list of retrieved documents."""
      return process_output


# **Processor 2: Generation**
# This processor takes the query and retrieved documents from the context and generates an answer.
# We'll use a placeholder function for the language model.

class GenerationProcessor:
  def __init__(self):
    # In a real scenario, you would load a language model here (e.g., from transformers library or OpenAI API)
    pass

  def generate_answer_with_llm(self, query, documents):
    """
    Placeholder for LLM call.
    In reality, you'd format the prompt with query and documents and call an API/model.
    """
    doc_texts = "\n".join([doc['text'] for doc in documents])
    if not doc_texts:
        return "Could not find relevant information."

    # Simple placeholder logic
    if "capital of France" in query.lower() and "Paris" in doc_texts:
        return "Based on the information, the capital of France is Paris."
    elif "Eiffel Tower" in query.lower() and "Paris" in doc_texts:
        return "The Eiffel Tower is in Paris."
    elif "Mona Lisa" in query.lower() and "Louvre" in doc_texts and "Paris" in doc_texts:
        return "The Mona Lisa is in the Louvre Museum in Paris."
    elif "capital of Italy" in query.lower() and "Rome" in doc_texts:
         return "Based on the information, the capital of Italy is Rome."
    else:
        return f"Based on the provided information:\n{doc_texts}\n\nAnswer to '{query}': This is a placeholder answer based on the docs."


  def process(self, input_value, context: RagContext):
    """
    Generates an answer based on the query and retrieved documents in the context.
    input_value is not used here.
    """
    if context.query is None or not context.retrieved_docs:
      # We can still attempt generation, but it might be less effective
      print("Warning: Generating answer without query or retrieved docs in context.")


    # Generate the answer using the LLM placeholder
    generated_answer = self.generate_answer_with_llm(context.query, context.retrieved_docs)

    # The "result" of this step is the generated answer
    result = generated_answer

    # Update the context with the generated answer
    updated_context = RagContext(
        query=context.query,
        retrieved_docs=context.retrieved_docs, # Keep retrieved docs
        generated_answer=generated_answer
    )

    return result, updated_context

  def get_result(self, process_output, context):
      """The result of this step is the generated answer."""
      return process_output


# **3. Building the Agent Workflow using MCP**

# An agent orchestrates the execution of different processors, managing the context flow between them.

class RagAgent:
  def __init__(self, retrieval_processor, generation_processor):
    self.retrieval_processor = retrieval_processor
    self.generation_processor = generation_processor
    self.current_context = None # The agent holds the persistent context

  def initialize_session(self):
    """Starts a new interaction session by resetting the context."""
    self.current_context = RagContext()
    print("Agent session initialized.")

  def ask_question(self, query):
    """Processes a user question using the RAG workflow."""
    if self.current_context is None:
        print("No active session. Initializing a new one.")
        self.initialize_session()

    # --- Step 1: Set the query in the context ---
    # This is a bit different - the *agent* updates the context directly to start.
    # Or, you could have a separate "Query Setter" processor. Let's do direct for simplicity.
    self.current_context.query = query
    print(f"\nAgent received query: '{query}'")
    print(f"Current context after setting query: {self.current_context}")

    # --- Step 2: Perform Retrieval ---
    print("\nAgent performing retrieval...")
    retrieval_result, updated_context_after_retrieval = self.retrieval_processor.process(None, self.current_context)
    self.current_context = updated_context_after_retrieval # Update agent's context
    retrieved_docs = self.retrieval_processor.get_result(retrieval_result, self.current_context)
    print(f"Retrieval complete. Found {len(retrieved_docs)} documents.")
    # print(f"Retrieved Docs: {retrieved_docs}") # Uncomment to see docs
    print(f"Current context after retrieval: {self.current_context}")


    # --- Step 3: Perform Generation ---
    print("\nAgent performing generation...")
    generation_result, updated_context_after_generation = self.generation_processor.process(None, self.current_context)
    self.current_context = updated_context_after_generation # Update agent's context
    final_answer = self.generation_processor.get_result(generation_result, self.current_context)
    print("Generation complete.")
    print(f"Current context after generation: {self.current_context}")


    # --- Step 4: Provide the Final Result ---
    print("\n--- Final Answer ---")
    print(final_answer)
    print("--------------------\n")

    return final_answer

  def get_current_context(self):
      return self.current_context


# **4. Running the RAG Agent Example**

# Instantiate the processors
retrieval_proc = RetrievalProcessor(corpus=corpus)
generation_proc = GenerationProcessor() # Placeholder LLM

# Instantiate the agent
rag_agent = RagAgent(retrieval_processor=retrieval_proc, generation_processor=generation_proc)

# Initialize a session
rag_agent.initialize_session()

# Ask a question
rag_agent.ask_question("What is the capital of France?")

# Ask another question - the context was reset in initialize_session
rag_agent.ask_question("Where is the Eiffel Tower?")

# Let's try asking a question that uses information from multiple documents or is slightly outside the simple rules
# rag_agent.ask_question("Tell me about the Mona Lisa location.")

# Examine the context after the last question
print("\nAgent's final context after last question:")
print(rag_agent.get_current_context())


# **5. Extending to More Complex Agentic Workflows**

# The MCP pattern is even more powerful for complex agents that might:
# - Use multiple tools (web search, calculator, API calls)
# - Engage in multi-turn conversations
# - Plan sequences of actions
# - Reflect on previous steps

# **Example: A Simple Multi-step Agent**

# Let's imagine an agent that needs to find information and then potentially perform a calculation.

# Define a new context that can hold more information
class MultiStepContext:
    def __init__(self, query=None, info_found=None, calculated_value=None, current_task="find_info"):
        self.query = query
        self.info_found = info_found # e.g., numbers extracted from text
        self.calculated_value = calculated_value
        self.current_task = current_task # State variable to guide the agent's next step

    def __repr__(self):
        return f"MultiStepContext(query='{self.query}', task='{self.current_task}', info_found={self.info_found}, calculated={self.calculated_value is not None})"

# Define a processor to find information (similar to Retrieval, but could extract specific data)
class InfoFinderProcessor:
    def process(self, input_value, context: MultiStepContext):
        """Simulates finding numerical information related to the query."""
        if context.query is None:
            return None, context # Cannot proceed without a query

        found_info = None
# Dummy logic: if query asks about "items" and mentions a number
        match = re.search(r'how many (\w+) are there with (\d+)', context.query)
        if match:
            item_type = match.group(1)
            number = int(match.group(2))
            print(f"  InfoFinder: Found query about '{item_type}' with number '{number}'.")
            # Simulate finding related info - let's say we find another number
            found_info = [number, number * 2] # Example: found the input number and double it

# Update context
        updated_context = MultiStepContext(
            query=context.query,
            info_found=found_info,
            calculated_value=context.calculated_value,
            current_task="calculate" if found_info is not None else "failed_info_find" # Update task based on finding info
        )
        return found_info, updated_context # Result of this step is the found info

    def get_result(self, process_output, context):
        return process_output

# Define a processor to perform a calculation
class CalculatorProcessor:
    def process(self, input_value, context: MultiStepContext):
        """Simulates a calculation based on info_found in the context."""
        if context.info_found is None or not isinstance(context.info_found, list) or len(context.info_found) < 2:
            print("  Calculator: Not enough information to calculate.")
            return None, context # Cannot calculate

        try:
            num1 = context.info_found[0]
            num2 = context.info_found[1]
            calculated_value = num1 + num2 # Simple calculation
            print(f"  Calculator: Calculating {num1} + {num2} = {calculated_value}")

            # Update context
            updated_context = MultiStepContext(
                query=context.query,
                info_found=context.info_found,
                calculated_value=calculated_value,
                current_task="report_result" # Move to reporting stage
            )
            return calculated_value, updated_context # Result is the calculated value

        except Exception as e:
            print(f"  Calculator Error: {e}")
            # Update context to reflect failure
            updated_context = MultiStepContext(
                query=context.query,
                info_found=context.info_found,
                calculated_value=None,
                current_task="calculation_failed"
            )
            return None, updated_context

    def get_result(self, process_output, context):
        return process_output

# Define a processor to report the final result
class ReporterProcessor:
    def process(self, input_value, context: MultiStepContext):
        """Generates a final report based on the context state."""
        final_report = "Processing complete."
        if context.calculated_value is not None:
            final_report = f"Based on your query, the calculated value is: {context.calculated_value}"
        elif context.info_found is not None:
             final_report = f"Found some information but couldn't complete the calculation. Information found: {context.info_found}"
        else:
             final_report = "Could not find relevant information or perform calculation."

# Context remains the same for the final reporting step, or you could add a 'finished' flag
        updated_context = MultiStepContext(
             query=context.query,
             info_found=context.info_found,
             calculated_value=context.calculated_value,
             current_task="finished" # Mark as finished
        )

        return final_report, updated_context # Result is the final report

    def get_result(self, process_output, context):
        return process_output


# Build a simple Multi-step Agent orchestrator
class MultiStepAgent:
    def __init__(self, info_finder, calculator, reporter):
        self.info_finder = info_finder
        self.calculator = calculator
        self.reporter = reporter
        self.current_context = None

    def initialize_session(self, query):
        """Starts a new multi-step session."""
        self.current_context = MultiStepContext(query=query, current_task="find_info")
        print(f"Agent session initialized for query: '{query}'")
        print(f"Initial context: {self.current_context}")

    def run_next_step(self):
        """Runs the next appropriate processor based on the current context's task."""
        if self.current_context is None:
            print("Agent not initialized. Call initialize_session first.")
            return None

        current_task = self.current_context.current_task
        print(f"\nAgent executing step: {current_task}")

        result = None
        updated_context = self.current_context # Default to no change if task not matched

        if current_task == "find_info":
            result, updated_context = self.info_finder.process(None, self.current_context)
            step_result = self.info_finder.get_result(result, updated_context)
            print(f"  Step result (Info Finder): {step_result}")

        elif current_task == "calculate":
             result, updated_context = self.calculator.process(None, self.current_context)
             step_result = self.calculator.get_result(result, updated_context)
             print(f"  Step result (Calculator): {step_result}")

        elif current_task == "report_result" or current_task == "failed_info_find" or current_task == "calculation_failed":
             result, updated_context = self.reporter.process(None, self.current_context)
             step_result = self.reporter.get_result(result, updated_context)
             print(f"  Step result (Reporter): {step_result}")
             print("\n--- Multi-step Process Finished ---")
             print(f"Final Outcome: {step_result}")
             # We might not update the context if it's the final step, or mark it finished
             self.current_context = updated_context # Update context to mark 'finished'
             return step_result # Return the final result

        else:
            print(f"Agent reached an unknown task state: {current_task}")
            # You might want to handle errors or loop detection here

        self.current_context = updated_context # Update the agent's context for the next step
        print(f"Context after step: {self.current_context}")

# Return the step result and indicate if finished
        return step_result # Return result of the just-completed step


    def run_until_finished(self):
        """Runs the agent step-by-step until it reaches a 'finished' state."""
        if self.current_context is None:
            print("Agent not initialized. Call initialize_session first.")
            return None

        last_result = None
        while self.current_context.current_task != "finished" and \
              "failed" not in self.current_context.current_task: # Also stop on failure states
            step_output = self.run_next_step()
            if self.current_context.current_task == "finished" or "failed" in self.current_context.current_task:
                 last_result = step_output # Capture the final output
                 break # Exit loop if finished or failed
            # Optional: Add a safety break for infinite loops
            # step_counter += 1
            # if step_counter > MAX_STEPS: break


        if self.current_context.current_task == "finished":
             print("\nAgent successfully finished.")
             return last_result # Return the final reported result
        else:
             print(f"\nAgent finished in state: {self.current_context.current_task}")
             if last_result:
                 print(f"Last step output: {last_result}")
             return None # Indicate failure or incomplete state


# **6. Running the Multi-step Agent Example**

# Instantiate processors
info_finder_proc = InfoFinderProcessor()
calculator_proc = CalculatorProcessor()
reporter_proc = ReporterProcessor()

# Instantiate the agent
multi_step_agent = MultiStepAgent(info_finder=info_finder_proc, calculator=calculator_proc, reporter=reporter_proc)

# Run a query through the agent workflow
query1 = "Can you calculate the sum of two numbers if there are 5 items and 10 items?"
multi_step_agent.initialize_session(query1)
final_output1 = multi_step_agent.run_until_finished()

print("\n--- End of Agent Run 1 ---\n")
print(f"Final context: {multi_step_agent.get_current_context()}")


# Run another query that might not trigger the calculation step
query2 = "Tell me about cats."
multi_step_agent.initialize_session(query2)
final_output2 = multi_step_agent.run_until_finished()

print("\n--- End of Agent Run 2 ---\n")
print(f"Final context: {multi_step_agent.get_current_context()}")


# **7. Conclusion**

# The Model Context Protocol (MCP), used here as a conceptual pattern, provides a robust way to structure
# complex AI workflows, particularly for RAG and agentic systems.

# By defining a clear context object and modular processors that update this context, we gain:
# - **Modularity:** Processors are self-contained and reusable.
# - **Statefulness:** The context preserves information across steps.
# - **Observability:** We can inspect the context at any point to understand the agent's state.
# - **Flexibility:** Easily add or change processors and orchestrate complex sequences of operations.

# This pattern is foundational for building sophisticated AI agents that can interact,
# use tools, and maintain coherence over multiple steps.



In [ ]:
# An example on SDLC using RAG and Agentic AI and use MCP

# Define the context structure for our SDLC agent
class SdlcContext:
    def __init__(self, requirement=None, retrieved_guidelines=None, initial_plan=None, generated_code_snippet=None, review_status=None, current_stage="requirements"):
        self.requirement = requirement
        self.retrieved_guidelines = retrieved_guidelines if retrieved_guidelines is not None else []
        self.initial_plan = initial_plan
        self.generated_code_snippet = generated_code_snippet
        self.review_status = review_status # e.g., "pending", "approved", "rejected"
        self.current_stage = current_stage # Tracks the current stage of the SDLC process

    def __repr__(self):
        return (f"SdlcContext(stage='{self.current_stage}', requirement='{self.requirement[:30]}...', "
                f"num_guidelines={len(self.retrieved_guidelines)}, plan_set={self.initial_plan is not None}, "
                f"code_set={self.generated_code_snippet is not None}, review='{self.review_status}')")


# Define Processors for each stage

# Processor 1: Requirements Gathering (Implicit - agent sets the initial context)
# This isn't a separate processor that takes context -> new context.
# The agent sets the initial requirement in the context.

# Processor 2: Retrieve Guidelines (RAG Component)
class GuidelineRetrievalProcessor:
    def __init__(self, guideline_corpus):
        self.guideline_corpus = guideline_corpus # Our "document store" for RAG

    def process(self, input_value, context: SdlcContext):
        """
        Retrieves relevant guidelines based on the requirement in the context.
        input_value is not used here.
        """
        if context.requirement is None:
            print("  GuidelineRetrieval: No requirement in context.")
            return None, context # Cannot retrieve without a requirement

        # Simple keyword matching for retrieval (replace with vector search/FAISS in real RAG)
        requirement_keywords = set(context.requirement.lower().split())
        retrieved_guidelines = []
        for guideline in self.guideline_corpus:
            if any(keyword in guideline['text'].lower() for keyword in requirement_keywords):
                retrieved_guidelines.append(guideline)

        print(f"  GuidelineRetrieval: Retrieved {len(retrieved_guidelines)} guidelines.")

        # Update context with retrieved guidelines and move to the next stage
        updated_context = SdlcContext(
            requirement=context.requirement,
            retrieved_guidelines=retrieved_guidelines,
            initial_plan=context.initial_plan,
            generated_code_snippet=context.generated_code_snippet,
            review_status=context.review_status,
            current_stage="planning" # Move to planning stage
        )

        # The result of this step is the retrieved guidelines
        result = retrieved_guidelines
        return result, updated_context

    def get_result(self, process_output, context):
         """The result is the list of retrieved documents."""
         return process_output


# Processor 3: Planning/Initial Design (Agentic Component)
class PlanningProcessor:
    def process(self, input_value, context: SdlcContext):
        """
        Generates an initial plan based on the requirement and retrieved guidelines.
        input_value is not used here.
        """
        if context.requirement is None:
            print("  Planning: No requirement in context.")
            # Fail or move to a different error stage
            updated_context = SdlcContext(
                 requirement=context.requirement,
                 retrieved_guidelines=context.retrieved_guidelines,
                 current_stage="planning_failed" # Mark as failed
            )
            return None, updated_context

        print("  Planning: Generating initial plan...")
        # Simple placeholder logic for plan generation using requirement and guidelines
        plan_steps = []
        plan_steps.append(f"Requirement: {context.requirement}")
        plan_steps.append("\nConsidered Guidelines:")
        if context.retrieved_guidelines:
            for i, guideline in enumerate(context.retrieved_guidelines):
                 plan_steps.append(f"- Guideline {guideline['id']}: {guideline['text'][:50]}...")
        else:
            plan_steps.append("- No specific guidelines retrieved.")

        plan_steps.append("\nInitial Plan/Pseudocode:")
        # More sophisticated LLM call would go here
        if "add numbers" in context.requirement.lower():
             plan_steps.append("1. Define a function that accepts a list of numbers.")
             plan_steps.append("2. Initialize sum to 0.")
             plan_steps.append("3. Iterate through the list, adding each number to the sum.")
             plan_steps.append("4. Return the sum.")
        elif "fetch data" in context.requirement.lower():
             plan_steps.append("1. Define a function to fetch data from a source (e.g., API, database).")
             plan_steps.append("2. Handle potential connection errors.")
             plan_steps.append("3. Parse the received data.")
             plan_steps.append("4. Return the processed data.")
        else:
             plan_steps.append("Basic plan based on requirement.")

        initial_plan = "\n".join(plan_steps)
        print(f"  Planning: Plan generated.")

        # Update context with the generated plan and move to the next stage
        updated_context = SdlcContext(
            requirement=context.requirement,
            retrieved_guidelines=context.retrieved_guidelines,
            initial_plan=initial_plan,
            generated_code_snippet=context.generated_code_snippet,
            review_status=context.review_status,
            current_stage="code_generation" # Move to code generation stage
        )

        # The result of this step is the generated plan
        result = initial_plan
        return result, updated_context

    def get_result(self, process_output, context):
        """The result is the generated plan."""
        return process_output

# Processor 4: Code Generation (Placeholder)
class CodeGenerationProcessor:
     def process(self, input_value, context: SdlcContext):
         """
         Simulates generating code based on the plan in the context.
         input_value is not used here.
         """
         if context.initial_plan is None:
             print("  CodeGeneration: No plan in context.")
             updated_context = SdlcContext(
                 requirement=context.requirement,
                 retrieved_guidelines=context.retrieved_guidelines,
                 initial_plan=context.initial_plan,
                 current_stage="code_generation_failed" # Mark as failed
             )
             return None, updated_context

         print("  CodeGeneration: Simulating code generation...")
         # Simple placeholder logic - just wrap the plan in comment blocks
         generated_code = f"# Generated Code Snippet\n# Based on plan:\n'''\n{context.initial_plan}\n'''\n\n# Actual code would be here...\npass"
         print("  CodeGeneration: Code snippet generated.")

         # Update context with the generated code and move to the next stage
         updated_context = SdlcContext(
             requirement=context.requirement,
             retrieved_guidelines=context.retrieved_guidelines,
             initial_plan=context.initial_plan,
             generated_code_snippet=generated_code,
             review_status="pending", # Set review status
             current_stage="review" # Move to review stage
         )

         # The result is the generated code
         result = generated_code
         return result, updated_context

     def get_result(self, process_output, context):
         """The result is the generated code snippet."""
         return process_output

# Processor 5: Review/Validation (Placeholder)
class ReviewProcessor:
     def process(self, input_value, context: SdlcContext):
         """
         Simulates a review of the generated code.
         input_value could be external feedback, but here it's unused.
         """
         if context.generated_code_snippet is None:
              print("  Review: No code snippet in context to review.")
              updated_context = SdlcContext(
                 requirement=context.requirement,
                 retrieved_guidelines=context.retrieved_guidelines,
                 initial_plan=context.initial_plan,
                 generated_code_snippet=context.generated_code_snippet,
                 review_status="not_applicable",
                 current_stage="review_skipped" # Or failed
              )
              return None, updated_context


         print("  Review: Simulating code review...")
         # Simple placeholder logic for review decision
         review_decision = "approved" # Assume approved for this example
         review_comments = "Looks good based on plan and guidelines."

         # Simulate a case where review fails based on keyword in requirement
         if "complex error handling" in context.requirement.lower():
              review_decision = "rejected"
              review_comments = "Review failed: Need more robust error handling as per requirement."


         print(f"  Review: Review status set to '{review_decision}'.")

         # Update context with the review status and move to the final stage
         updated_context = SdlcContext(
             requirement=context.requirement,
             retrieved_guidelines=context.retrieved_guidelines,
             initial_plan=context.initial_plan,
             generated_code_snippet=context.generated_code_snippet,
             review_status=review_decision,
             # Based on review, agent could decide next steps (e.g., "revise_code" or "deploy")
             current_stage="finished" if review_decision == "approved" else "needs_revision"
         )

         # The result of this step is the review status and comments
         result = {"status": review_decision, "comments": review_comments}
         return result, updated_context

     def get_result(self, process_output, context):
         """The result is the review outcome."""
         return process_output


# SDLC Agent Orchestrator using MCP
class SdlcAgent:
     def __init__(self, retrieval_processor, planning_processor, code_gen_processor, review_processor):
         self.retrieval_processor = retrieval_processor
         self.planning_processor = planning_processor
         self.code_gen_processor = code_gen_processor
         self.review_processor = review_processor
         self.current_context = None

     def start_feature_development(self, requirement):
         """Starts a new SDLC process for a feature requirement."""
         # Initialize the context with the requirement
         self.current_context = SdlcContext(requirement=requirement, current_stage="requirements")
         print(f"SDLC Agent: Starting process for requirement: '{requirement}'")
         print(f"Initial context: {self.current_context}")

     def run_next_stage(self):
         """Runs the next appropriate processor based on the current context's stage."""
         if self.current_context is None:
             print("SDLC Agent not initialized. Call start_feature_development first.")
             return None

         current_stage = self.current_context.current_stage
         print(f"\nSDLC Agent: Executing stage: {current_stage}")

         result = None
         updated_context = self.current_context # Default

         try:
             if current_stage == "requirements":
                 # This stage is primarily setting the initial context, which the agent does in start_feature_development
                 # We immediately transition to the next stage.
                 print("  SDLC Agent: Requirements stage complete (context initialized). Transitioning to retrieval.")
                 updated_context = SdlcContext( # Create new context to update stage
                      requirement=self.current_context.requirement,
                      retrieved_guidelines=self.current_context.retrieved_guidelines,
                      initial_plan=self.current_context.initial_plan,
                      generated_code_snippet=self.current_context.generated_code_snippet,
                      review_status=self.current_context.review_status,
                      current_stage="retrieval"
                 )
                 step_result = "Requirements set."

             elif current_stage == "retrieval":
                 result, updated_context = self.retrieval_processor.process(None, self.current_context)
                 step_result = self.retrieval_processor.get_result(result, updated_context)
                 print(f"  Step result (Retrieval): Found {len(step_result)} guidelines.")

             elif current_stage == "planning":
                  result, updated_context = self.planning_processor.process(None, self.current_context)
                  step_result = self.planning_processor.get_result(result, updated_context)
                  print(f"  Step result (Planning): Plan generated.")
                  # print(step_result) # Uncomment to see the plan

             elif current_stage == "code_generation":
                  result, updated_context = self.code_gen_processor.process(None, self.current_context)
                  step_result = self.code_gen_processor.get_result(result, updated_context)
                  print(f"  Step result (Code Generation): Code snippet generated.")
                  # print(step_result) # Uncomment to see the code

             elif current_stage == "review":
                  result, updated_context = self.review_processor.process(None, self.current_context)
                  step_result = self.review_processor.get_result(result, updated_context)
                  print(f"  Step result (Review): Status='{step_result['status']}', Comments='{step_result['comments']}'")
                  # Note: The agent logic here is simple. A real agent might decide to loop back
                  # to 'code_generation' if review_status is 'rejected'.

             elif current_stage == "finished":
                 print("  SDLC Agent: Process is already finished.")
                 step_result = "Process already finished."
                 return step_result # Stop processing

             elif "failed" in current_stage or "skipped" in current_stage or "needs_revision" in current_stage:
                 print(f"  SDLC Agent: Process stopped in state: {current_stage}")
                 step_result = f"Process ended in state: {current_stage}"
                 return step_result # Stop processing on failure/terminal states

             else:
                 print(f"  SDLC Agent: Unknown stage: {current_stage}. Stopping.")
                 updated_context = SdlcContext( # Set a failed state
                     requirement=self.current_context.requirement,
                     current_stage=f"unknown_stage_error_{current_stage}"
                 )
                 step_result = f"Unknown stage error: {current_stage}"

             self.current_context = updated_context # Update the agent's context

             print(f"Context after stage: {self.current_context}")
             return step_result # Return the result of the completed stage

         except Exception as e:
             print(f"SDLC Agent Error during stage '{current_stage}': {e}")
             # Update context to reflect an error state
             self.current_context = SdlcContext(
                 requirement=self.current_context.requirement if self.current_context else None,
                 current_stage=f"process_error_{current_stage}"
             )
             print(f"Context after error: {self.current_context}")
             return f"Error occurred during stage '{current_stage}'"


     def run_full_sdlc_cycle(self, requirement):
         """Runs all defined stages for a requirement until completion or failure."""
         self.start_feature_development(requirement)

         last_result = None
         # Define terminal states explicitly or implicitly
         terminal_states = ["finished", "planning_failed", "code_generation_failed",
                            "review_skipped", "review_failed", "unknown_stage_error",
                            "process_error", "needs_revision"]

         while self.current_context.current_stage not in terminal_states:
              last_result = self.run_next_stage()
              if last_result is not None and "Error occurred" in last_result:
                  print("Stopping due to error.")
                  break # Stop on process errors

              # Add a safeguard against infinite loops (e.g., if stage transitions are broken)
              # For this simple linear flow, it's less critical, but good practice.
              # if stage_counter > MAX_SDLC_STAGES: break


         print("\n--- SDLC Process Complete ---")
         print(f"Final stage reached: {self.current_context.current_stage}")
         print(f"Final context: {self.current_context}")

         if self.current_context.current_stage == "finished":
              print("\nFeature development process successfully completed!")
              return last_result # The result from the review stage
         elif self.current_context.current_stage == "needs_revision":
              print("\nFeature development needs revision based on review.")
              # A real agent might trigger another cycle starting from code generation
              return "Needs Revision"
         else:
              print("\nFeature development process ended prematurely or with failure.")
              return f"Process ended in state: {self.current_context.current_stage}"


     def get_current_context(self):
         return self.current_context


# **Example Guideline Corpus (for RAG)**
sdlc_guidelines_corpus = [
    {"id": 101, "text": "All functions should include docstrings explaining their purpose, arguments, and return values."},
    {"id": 102, "text": "Use snake_case for variable and function names in Python."},
    {"id": 103, "text": "Prefer using standard library functions where available."},
    {"id": 104, "text": "Implement robust error handling using try...except blocks for potential failures like network requests or invalid input."},
    {"id": 105, "text": "Ensure that unit tests are written for all new code."},
    {"id": 201, "text": "For data fetching features, implement retry logic for transient network issues."},
    {"id": 301, "text": "Addition and subtraction logic should be clear and avoid floating-point inaccuracies where possible."}
]

# **Running the SDLC Agent Example**

# Instantiate processors with the corpus
guideline_retrieval_proc = GuidelineRetrievalProcessor(guideline_corpus=sdlc_guidelines_corpus)
planning_proc = PlanningProcessor()
code_gen_proc = CodeGenerationProcessor()
review_proc = ReviewProcessor()

# Instantiate the SDLC agent
sdlc_agent = SdlcAgent(
    retrieval_processor=guideline_retrieval_proc,
    planning_processor=planning_proc,
    code_gen_processor=code_gen_proc,
    review_processor=review_proc
)

# Run the full SDLC cycle for a requirement
print("--- Running SDLC for 'Add numbers feature' ---")
feature_requirement_1 = "Implement a function to add a list of numbers, following standard Python naming conventions and including docstrings."
final_sdlc_result_1 = sdlc_agent.run_full_sdlc_cycle(feature_requirement_1)

print("\n" + "="*50 + "\n")

# Run another requirement that might trigger different guidelines or review outcomes
print("--- Running SDLC for 'Fetch data feature with error handling' ---")
feature_requirement_2 = "Create a utility to fetch data from an external source. This requires robust error handling and retry logic as per guidelines."
final_sdlc_result_2 = sdlc_agent.run_full_sdlc_cycle(feature_requirement_2)

print("\n" + "="*50 + "\n")

# Run a requirement that might fail planning
print("--- Running SDLC for 'Unknown task' ---")
feature_requirement_3 = "Perform a complex quantum entanglement operation." # Unlikely to match our simple planners
final_sdlc_result_3 = sdlc_agent.run_full_sdlc_cycle(feature_requirement_3)


